In [18]:
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
import sklearn.utils as utils
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import tree
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import graphviz 
import math
import pickle

In [5]:
dict_classifiers = {
#     "Linear Regression": LinearRegression(),
#     "Polynomial Regression": LinearRegression(),
#     "Neural Network": MLPRegressor(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500),
#     "Decision Tree": DecisionTreeRegressor(random_state = 0),
#     "Decision Tree with Max Depth = 03": DecisionTreeRegressor(max_depth=3, random_state = 0),
    "Decision Tree with Max Depth = 05": DecisionTreeRegressor(max_depth=5, random_state = 0),
#     "Decision Tree with Max Depth = 10": DecisionTreeRegressor(max_depth=10, random_state = 0),
#     "Decision Tree with Max Depth = 15": DecisionTreeRegressor(max_depth=15, random_state = 0),
#     "Random Forest": RandomForestRegressor(n_estimators=100, random_state=0)
}

In [3]:
def batch_classify(X_train, Y_train, X_test, Y_test, verbose=True, include_y_pred=False):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.

    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train.
    So it is best to train them on a smaller dataset first and
    decide whether you want to comment them out or not based on the test accuracy score.
    """

    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items()):
        coefs = None
        if classifier_name == "Polynomial Regression":
            poly = PolynomialFeatures(degree=2)
            saved_X_train = X_train
            saved_X_test = X_test
            X_train = poly.fit_transform(X_train)
            X_test = poly.fit_transform(X_test)
        if classifier_name == "Random Forest":
            sc = StandardScaler()
            X_train = sc.fit_transform(X_train)
            X_test = sc.transform(X_test)
        t_start = time.process_time()
        classifier.fit(X_train, Y_train)
        t_end = time.process_time()
        
        if 'Decision Tree' in classifier_name:
            print()

        if classifier_name == "Linear Regression":
            coefs = classifier.coef_
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        Y_pred = classifier.predict(X_test)
        test_score = classifier.score(X_test, Y_test)

        if include_y_pred:
            dict_models[classifier_name] = {'model': classifier_name, 'train_score': train_score,
                                            'test_score': test_score, 'train_time': t_diff, 'y_pred': Y_pred}
        else:
            dict_models[classifier_name] = {'model': classifier_name, 'train_score': train_score,
                                            'test_score': test_score, 'train_time': t_diff, 'coef':coefs}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
        if classifier_name == "Polynomial Regression":
            X_train = saved_X_train
            X_test = saved_X_test
    return dict_models

In [4]:
def scores_calculator(classifiers, model):
    score_dict = {}
    for classifier in classifiers:
        Y_pred_bin = model[classifier]['y_pred'] > 2
        Y_test_bin = Y_test > 2
        true_pos_count = 0
        true_neg_count = 0
        false_pos_count = 0
        false_neg_count = 0
        for i in range(len(Y_pred_bin)):
            if Y_pred_bin[i] == Y_test_bin[i]:
                if Y_pred_bin[i] == True:
                    true_pos_count += 1
                else:
                    true_neg_count += 1
            else:
                if Y_pred_bin[i] == True:
                    false_pos_count += 1
                else:
                    false_neg_count += 1
                    
        accuracy = (true_pos_count + true_neg_count) / len(Y_pred_bin)
        if (true_pos_count + false_neg_count) != 0:
            recall = true_pos_count / (true_pos_count + false_neg_count)
        else:
            recall = None
        if (true_pos_count + false_pos_count) != 0:
            precision = true_pos_count / (true_pos_count + false_pos_count)
        else:
            precision = None
                    
        score_dict[classifier] = {'Accuracy': accuracy, 
                                 'Recall': recall,
                                 'Precision': precision}
        
    return score_dict

In [6]:
files = os.listdir('../Data/rmsds')
first = True
data = pd.read_csv("../Data/rmsds/5HT2B_rmsds.csv")
for file in files:
    if file[-4:] == '.csv' and file != '5HT2B_rmsds.csv':
        fileData = pd.read_csv("../Data/rmsds/" + file)
        data = pd.concat([data, fileData], sort=False)
data = data[data['secondary structure'] != -1]
data = data[data['complete rmsd'] < 30]
prots = sorted(data['protein'].unique())
test_results = []
test_scores = []

test_prot = 'MAPK14'
train_data = data[data['protein'] != test_prot]
test_data = data[data['protein'] == test_prot]

X_train = train_data.drop(['protein', 'start ligand', 'target ligand', 'name', 'num', 'complete rmsd', 'backbone rmsd', 
                           'sidechain rmsd'], axis=1).values
Y_train = train_data['complete rmsd'].values 

X_test = test_data.drop(['protein', 'start ligand', 'target ligand', 'name', 'num', 'complete rmsd', 'backbone rmsd', 
                         'sidechain rmsd'], axis=1).values
Y_test = test_data['complete rmsd'].values 

X_true = X_train[Y_train > 2]
Y_true = Y_train[Y_train > 2]

X_train_expanded_50 = np.append(X_train, X_true, axis=0)
Y_train_expanded_50 = np.append(Y_train, Y_true, axis=0)

for i in range(4):
    X_train_expanded_50 = np.append(X_train_expanded_50, X_true, axis=0)
    Y_train_expanded_50 = np.append(Y_train_expanded_50, Y_true, axis=0)

dict_models = batch_classify(X_train_expanded_50, Y_train_expanded_50, X_test, Y_test, include_y_pred=True)
test_results.append(dict_models)
test_scores.append(scores_calculator(list(dict_classifiers.keys()), dict_models))


trained Decision Tree with Max Depth = 05 in 4.38 s


In [37]:
y_pred_data = test_data.assign(y_pred =  dict_models['Decision Tree with Max Depth = 05']['y_pred'])
y_pred_data.to_csv('../Data/MAPK14_y_pred')

In [34]:
y_pred_pair = y_pred_data[(y_pred_data['start ligand'] == '1KV1') & (y_pred_data['target ligand'] == '1YQJ')]
y_pred_pair.head()

,protein,start ligand,target ligand,name,num,bfactor,normalized bfactor,prev prev bfactor,prev bfactor,next bfactor,...,solvent accessibility,secondary structure,ligand similarity,ligand similarity ratio,ligand size difference,ligand size ratio,complete rmsd,backbone rmsd,sidechain rmsd,y_pred
0,MAPK14,1KV1,1YQJ,ALA,34,58.630000,1.958952,2.092002,2.089175,1.897189,...,78.870961,0,6,0.285714,-14,0.6,9.291662,8.113508,9.291662,4.134324
1,MAPK14,1KV1,1YQJ,TYR,35,57.938333,1.897189,1.958952,2.092002,1.270481,...,79.360734,0,6,0.285714,-14,0.6,12.981802,8.506663,12.981802,4.134324
2,MAPK14,1KV1,1YQJ,VAL,38,38.252857,0.139359,0.712829,1.270481,0.218279,...,16.900358,2,6,0.285714,-14,0.6,1.197541,1.139571,1.197541,2.602984
3,MAPK14,1KV1,1YQJ,ALA,51,29.968000,-0.600444,-0.336077,1.139135,-0.696832,...,4.173099,2,6,0.285714,-14,0.6,0.655514,0.635199,0.655514,1.594001
4,MAPK14,1KV1,1YQJ,VAL,52,28.888571,-0.696832,-0.600444,-0.336077,-0.264910,...,0.000000,2,6,0.285714,-14,0.6,0.495412,0.486634,0.495412,0.576466


In [35]:
y_pred_pair = y_pred_pair.sort_values('y_pred')[-6:].sort_values('num')
y_pred_pair

,protein,start ligand,target ligand,name,num,bfactor,normalized bfactor,prev prev bfactor,prev bfactor,next bfactor,...,solvent accessibility,secondary structure,ligand similarity,ligand similarity ratio,ligand size difference,ligand size ratio,complete rmsd,backbone rmsd,sidechain rmsd,y_pred
1,MAPK14,1KV1,1YQJ,TYR,35,57.938333,1.897189,1.958952,2.092002,1.270481,...,79.360734,0,6,0.285714,-14,0.6,12.981802,8.506663,12.981802,4.134324
17,MAPK14,1KV1,1YQJ,LEU,108,44.442500,0.692068,0.162946,-0.650271,1.672201,...,48.706479,0,6,0.285714,-14,0.6,0.881922,0.899822,0.881922,4.134324
18,MAPK14,1KV1,1YQJ,MET,109,55.418750,1.672201,0.692068,0.162946,1.614047,...,37.258056,0,6,0.285714,-14,0.6,2.709487,1.805649,2.709487,4.134324
19,MAPK14,1KV1,1YQJ,GLY,110,54.767500,1.614047,1.672201,0.692068,1.561764,...,66.584555,0,6,0.285714,-14,0.6,1.589971,1.097710,1.589971,4.134324
21,MAPK14,1KV1,1YQJ,ASP,112,54.860000,1.622307,1.561764,1.614047,1.668852,...,30.215413,0,6,0.285714,-14,0.6,1.251185,0.793083,1.251185,4.134324
31,MAPK14,1KV1,1YQJ,PHE,169,38.790000,0.187323,-0.538205,-1.124543,1.072021,...,89.412818,0,6,0.285714,-14,0.6,9.996831,7.009494,9.996831,4.134324
